In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

color_pal=sns.color_palette()

# Feature Selection


In [184]:
df = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv')
df_eval =pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv')

In [185]:
df['date']=pd.to_datetime(df['date'])
df['store']=df['store'].astype('category')
df['item']=df['item'].astype('category')

df_eval['date']=pd.to_datetime(df_eval['date'])
df_eval['store']=df_eval['store'].astype('category')
df_eval['item']=df_eval['item'].astype('category')


print(df.head())
print(df_eval.head())

        date store item  sales
0 2013-01-01     1    1     13
1 2013-01-02     1    1     11
2 2013-01-03     1    1     14
3 2013-01-04     1    1     13
4 2013-01-05     1    1     10
   id       date store item
0   0 2018-01-01     1    1
1   1 2018-01-02     1    1
2   2 2018-01-03     1    1
3   3 2018-01-04     1    1
4   4 2018-01-05     1    1


In [186]:


def create_features(df):
    df = df.copy()
#     df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofweek'] = df['date'].dt.dayofweek
#     df['dayofyear'] = df['date'].dt.dayofyear
#     week_data = df['date'].dt.isocalendar()
#     df['weekofyear'] = week_data['week'].astype(int)
#     df['quarter'] = df['date'].dt.quarter
#     df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
#     df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    return df


In [187]:
df=create_features(df)
df_eval = create_features(df_eval)
print(df.head())

        date store item  sales  month  year  dayofweek
0 2013-01-01     1    1     13      1  2013          1
1 2013-01-02     1    1     11      1  2013          2
2 2013-01-03     1    1     14      1  2013          3
3 2013-01-04     1    1     13      1  2013          4
4 2013-01-05     1    1     10      1  2013          5


In [191]:
baseline = df.sales.mean()

store_item_table = pd.pivot_table(df, index='store', columns='item',values='sales', aggfunc=np.mean)
monthly_sales = pd.pivot_table(df, index='month', values='sales', aggfunc=np.mean)
monthly_sales.sales/=baseline
day_of_week_sales = pd.pivot_table(df, index='dayofweek', values='sales', aggfunc=np.mean)
day_of_week_sales.sales/=baseline
year_sales = pd.pivot_table(df, index='year', values='sales', aggfunc=np.mean)
year_sales/=baseline

years = np.arange(2013, 2019)
annual_sales_avg = year_sales.values.squeeze()
weights = np.exp((years - 2018)/6)
annual_growth = np.poly1d(np.polyfit(years[:-1], annual_sales_avg, 2,w=weights[:-1]))

In [192]:
df_eval['sales']= 0
for index,row in df_eval.iterrows():
    day_of_week,month,year = row.date.dayofweek, row.date.month, row.date.year
    item, store = row['item'],row['store']
    base_sale = store_item_table.at[store,item]
    predicted_sale=base_sale*monthly_sales.at[month,'sales'] * day_of_week_sales.at[day_of_week,'sales']*annual_growth(year)
    df_eval.at[index, 'sales'] = predicted_sale.round()
    

print(df_eval.head())


   id       date store item  month  year  dayofweek  sales
0   0 2018-01-01     1    1      1  2018          0     12
1   1 2018-01-02     1    1      1  2018          1     14
2   2 2018-01-03     1    1      1  2018          2     15
3   3 2018-01-04     1    1      1  2018          3     16
4   4 2018-01-05     1    1      1  2018          4     17


In [190]:
submission=df_eval[['id','sales']]
print(submission.head())

   id  sales
0   0     12
1   1     14
2   2     14
3   3     15
4   4     16


In [ ]:
submission.to_csv('submission.csv', index=False)
